In [157]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm

In [149]:
url = 'https://www.drugs.com/alpha/a.html'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [87]:
count= len(soup.findAll('div')[5].findAll('li'))
print(count)

In [88]:
lst_of_links = []
for i in range(0,count):
    links = soup.findAll('div')[5].findAll('li')[i].find('a')['href']
    lst_of_links.append(f"https://www.drugs.com{links}")

In [159]:
common_drgs_lst = []
for lst_of_link in lst_of_links:
    url = lst_of_link
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    count_of_most_common = len(soup.findAll('ul')[5].findAll('li'))
    for i in range(0,count_of_most_common):
        common_drugs_link = soup.findAll('ul')[5].findAll('li')[i].find('a')['href']
        common_drug_name = soup.findAll('ul')[5].findAll('li')[i].find('a').text
        
        common_drgs_dict = {'link': f"https://www.drugs.com{common_drugs_link}",
                            'name': common_drug_name}


        common_drgs_lst.append(common_drgs_dict)

In [161]:
# common_drgs_lst

In [117]:
soup.findAll('ul')[5].findAll('li')[0].find('a')['href']

'/abilify.html'

## Data Cleaning

In [147]:
def clean_data(text):
    # Remove excess whitespace and newlines
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace/newlines with a single space
    text = text.strip()  # Remove leading/trailing whitespace
    
    # Optional: Remove known irrelevant sections (like Copyright)
    text = re.sub(r'Copyright.*', '', text)
    
    return text

In [170]:
common_drgs_lst[0]['name']

'Abilify'

In [177]:
common_drugs_pd = []
for i in tqdm(range(len(common_drgs_lst))):
    url = common_drgs_lst[i]['link']
    name = common_drgs_lst[i]['name']
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    text_data = clean_data(soup.text)
    common_drgs_dict = {'link': url,
                        'name': name,
                       'description': text_data}
    common_drugs_pd.append(common_drgs_dict)

100%|███████████████████████████████████████| 1217/1217 [08:27<00:00,  2.40it/s]


In [180]:
df_common_drugs = pd.DataFrame(common_drugs_pd)

## Save Data to CSV FILE

In [182]:
df_common_drugs.to_csv('onemed.csv', index= False
                      )